In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy.stats
import pandas as pd
import os
import pandas.api.types
import kego.plotting
import kego.ariel.metrics
import kego.plotting.axes_utils
import kego.plotting.histograms
import kego.plotting.utils_plotting
import kego.plotting.figures
import sklearn.model_selection
import sklearn.ensemble
import sklearn.linear_model

In [ ]:
FOLDER_DOWNLOADS = os.environ["PATH_CFOLDER"] + "Users/krist/Downloads/"
# FOLDER_COMPETITION = "/home/kristian/Projects/kego/data/ariel/competition/"
FOLDER_COMPETITION = os.environ["PATH_EFOLDER"] + "/ariel-data-challenge-2024/"
os.listdir(FOLDER_DOWNLOADS)[:2]

In [ ]:
!ls $FOLDER_COMPETITION/test_adc_info.csv

In [ ]:
X_submission = pd.read_csv(FOLDER_COMPETITION + "/test_adc_info.csv")
X_submission

In [ ]:
X = pd.read_csv(FOLDER_DOWNLOADS + "phases_step8000_nids673.csv").drop(
    columns=["Unnamed: 0"]
)
y = pd.read_csv(FOLDER_COMPETITION + "train_labels.csv")

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y.drop(columns=["planet_id"]), test_size=0.3, random_state=42
)

In [ ]:
model = sklearn.ensemble.HistGradientBoostingRegressor()
model = sklearn.linear_model.RidgeCV()
oof_pred = sklearn.model_selection.cross_val_predict(model, X_train, y_train, cv=5)

In [ ]:
final_model = model.fit(X_train, y_train)
y_prediction = final_model.predict(X_test)

In [ ]:
def score(truths, predictions):
    solution = truths
    predictions = np.clip(predictions, a_min=0, a_max=None)
    submission_values = pd.DataFrame(data=predictions)
    submission_sigmas = pd.DataFrame(
        data=np.ones_like(predictions) * np.mean(predictions) * 0.1
    )
    submission = pd.concat(
        (submission_values, submission_sigmas), axis=1, ignore_index=True
    )
    naive_mean = np.mean(truths)
    naive_sigma = np.std(truths).mean()
    sigma_true = naive_sigma
    print(f"{naive_mean=}")
    print(f"{naive_sigma=}")
    print(f"{sigma_true=}")
    return kego.ariel.metrics.score(
        solution=solution,
        submission=submission,
        row_id_column_name=None,
        naive_mean=naive_mean,
        naive_sigma=naive_sigma,
        sigma_true=sigma_true,
    )

In [ ]:
score(truths=y_train, predictions=oof_pred), score(
    truths=y_test, predictions=y_prediction
)

In [ ]:
n_show_columns = 5
for i in range(n_show_columns):
    figure, axes = kego.plotting.figures.create_figure_axes()
    kego.plotting.histograms.plot_histogram_2d(
        oof_pred[:, i],
        y_train.iloc[:, i].to_numpy(),
        label_x="pred",
        label_y="true",
        axes=axes,
        annotate_line="unity",
    )

In [ ]:
import kego.plotting.axes_utils
import kego.plotting.figures
import kego.plotting.utils_plotting


figure, axes = kego.plotting.figures.create_figure_axes(figure_size=(12, 10))
for index in range(len(y)):
    axes.plot(y.iloc[index, 1:])
    kego.plotting.axes_utils.set_axes(axes, labelrotation_x=90, ylim=[0, None])
    if index >= 20:
        break

In [ ]:
solution = pd.concat([ids, y], axis=1)
submission = pd.concat([ids, y * 0.79, y.add_prefix("sigma_") * 0.1], axis=1)
submission.shape, solution.shape, y.shape[1] * 2

In [ ]:
def retrieve_pred_true(submission, solution, row_id_column_name="planet_id"):
    solution = solution.copy()
    submission = submission.copy()
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    if submission.min().min() < 0:
        raise ValueError("Negative values in the submission")
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ValueError(f"Submission column {col} must be a number")
    n_wavelengths = len(solution.columns)
    y_pred = submission.iloc[:, :n_wavelengths].values
    if len(submission.columns) != n_wavelengths * 2:
        raise ValueError(
            "Wrong number of columns in the submission"
        )  # Set a non-zero minimum sigma pred to prevent division by zero errors.
    sigma_pred = np.clip(
        submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None
    )
    y_true = solution.values
    return y_pred, y_true, sigma_pred


naive_mean = np.mean(y)
naive_sigma = y.to_numpy().std()
sigma_true = solution.iloc[:, 1:].to_numpy().std()

y_pred, y_true, sigma_pred = retrieve_pred_true(
    submission=submission, solution=solution
)
gaussian_pred = scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred)
gaussian_true = scipy.stats.norm.logpdf(
    y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)
)

In [ ]:
import kego.ariel.metrics

data_full = y

kego.ariel.metrics.score(
    solution=solution,
    submission=submission,
    row_id_column_name="planet_id",
    naive_mean=naive_mean,
    naive_sigma=naive_sigma,
    sigma_true=sigma_true,
)

In [ ]:
submission.iloc[:, 1:]

In [ ]:
solution.iloc[:, 1:]

In [ ]:
import kego.plotting.axes_utils
import kego.plotting.histograms
import kego.plotting.utils_plotting


plot_types = ["histogram", "lineplot", "lineplot_logpdf_true", "lineplot_logpdf_pred"]
samples = range(5)
print(f"{len(samples)=}")
figure, axes_list, colorbar_list = kego.plotting.figures.create_axes_grid(
    n_columns=len(plot_types), n_rows=len(samples), spacing_x=0.0, spacing_y=0.0
)
for i_sample, sample in enumerate(samples):
    for i_type, _type in enumerate(plot_types):
        axes = axes_list[i_sample, i_type]
        data = y.iloc[sample, 1:]
        labelrotation_x = 0
        ylim = None
        title = None
        if _type == "histogram":
            kego.plotting.histograms.plot_histogram(
                data, axes=axes, alpha=0.2, xlim=[np.min(data_full), np.max(data_full)]
            )
            ylim = [0, 300]
        elif _type == "lineplot":
            axes.plot(y.iloc[sample, 1:])
            labelrotation_x = 90
        elif _type == "lineplot_logpdf_true":
            axes.plot(gaussian_true[:, sample])
        elif _type == "lineplot_logpdf_pred":
            axes.plot(gaussian_pred[:, sample])
        if i_sample == 0:
            title = _type
        kego.plotting.axes_utils.set_axes(
            axes, labelrotation_x=labelrotation_x, ylim=ylim, title=title
        )

In [ ]:
import kego.ariel.metrics

kego.ariel.metrics.score(
    solution=solution,
    submission=submission,
    row_id_column_name="planet_id",
    naive_mean=np.mean(data_full),
    naive_sigma=data_full.to_numpy().std(),
    sigma_true=1e-5,
)